In [ ]:
# Libraries
import pandas as pd
import numpy as np
import scipy as sp
import math 
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy import special
import os
from sklearn.metrics import r2_score

In [ ]:
# Reading multiple csv files

# Enter your path
path=r'C:\Users\malli\OneDrive\Desktop\SIMPLE\Pass2 - Beam waist data 5-26-22'

file_directory=[]

for filename in sorted(os.listdir(path)):
    if filename.endswith(".csv"): 
        print(filename)
        df = pd.read_csv(filename, delimiter=';', header=16,  usecols=[2, 3], names=['Time', 'Power'])
        file_directory.append(df)

In [ ]:
# define function for curve fitting - - error function
def model(x,x0,p_max,w):
    return (1/2)*p_max*(1+special.erf(math.sqrt(2)*(x-x0)/w))

# initial guesses for parameters for x0, p_max, w
pO=[100,30,50] 

ws=[]
        
for df in file_directory:
    length = len(df['Time'])
    x = np.arange(length)
    y = 10**3*df['Power']
    
    # fit curve 
    popt, pcov = curve_fit(model,x,y,pO) #popt -- optimal values of parameters
    
    #define the fitting function
    yp=model(x,popt[0],popt[1],popt[2])
    
    # can take printing these values out; however seeing p_max and x0 can serve as sanity check that it's fitting well
    print('x0 = %.2f s'% (popt[0]))
    print('Maximum power = {:.2f} mW'.format(popt[1]))
    print('Beam waist = {:.2f} s'.format(popt[2]))
    print('R^2 : %.5f'%(r2_score(y,yp)))
    
    w=popt[2]
    ws.append(w)
   
    # would be nice to find way to label each graph with z position in the loop
    plt.figure()
    plt.title('Data against error function for knife edge method')
    plt.xlabel('Time (s)')
    plt.ylabel('Optical power (mW)')
    plt.plot(x,y,label='Data')
    plt.plot(x,yp, label='Fit')
    #
    plt.legend()

In [ ]:
# Define waist: only helpful if we work in meters
def waist_dist(z, z0, w0):
    l = 1.064e-6 #wavelength of beam
    zr = sp.pi*w0**2/l #raleigh length
    return w0*sp.sqrt(1 + ((z-z0)/zr)**2) 

# Function we use if we are working in z=time
def waist_time(z,z0,w0,a):
    return w0*sp.sqrt(1 + (a*(z-z0)/w0**2)**2) 

# MUST change to match heights used 
z = np.linspace(4,11,8)

# initial guess for z0,w0,a
g1=[10,15, -150] 

popt, pcov = curve_fit(waist_time,z,ws,g1)

#define the fitting function
yp_w=waist_time(z,popt[0], popt[1],popt[2])
print('z0 = %.8f s'% (popt[0]))
print('w0 = %.5f s'% (popt[1]))
print('a = %.5f s'% (popt[2]))

# find R^2 value
print('R^2 : %.5f'%(r2_score(ws,yp_w)))
plt.figure()

#plt.title('Beam Waist using Knife Edge Method')
plt.xlabel('z: Time (s) -- fast')
plt.ylabel('w')

#Plot data
plt.plot(z,ws, label='from erf', marker=".")

#Plot the fitting function
plt.plot(z,yp_w, label='fit', marker=".")
plt.legend()